### 0. Load environment variables

In [21]:
import os, sys
from dotenv import load_dotenv
_ = load_dotenv('.env')
sys.path.extend(['../..', '..'])

### 1. Get OpenAI functions
This prompt is supposed to work with OpenAI Functions 

In [22]:
from utils.pydantic_models.agent_trajectory import Trajectory
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
trajectory_getting_func = convert_pydantic_to_openai_function(Trajectory)

### 2. Get an example of input variables for the prompt

In [23]:
task = "Which is bigger? BTS's oldest member's birth month vs. incumbent Korean president's birth month."
scratchpad = """Entering new AgentExecutor chain...
Thought: I need to find out the birth month of the oldest member of BTS and the birth month of the current Korean president. 

Action:
{
  "action": "GetFromYDC",
  "action_input": {
    "Query": "Oldest member of BTS birth month"
  }
}

The oldest member of BTS is Jin, and his birth month is December. He was born on December 4, 1992.Action:

Action:

{
  "action": "GetFromYDC",
  "action_input": {
    "Query": "Current Korean president birth month"
  }
}

The current President of South Korea, Yoon Suk Yeol, was born on 18 December 1960. So, his birth month is December.

Action:
{
  "action": "Final Answer",
  "action_input": "The birth month of the oldest member of BTS, Jin, and the birth month of the current Korean president, Yoon Suk Yeol, are both December. Therefore, neither is bigger as they are the same."
}


Finished chain."""

### 3. Fetch the prompt and create a chain (using LCEL)

In [24]:
from typing import Literal
from langchain import hub
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from utils.custom_output_parsers.functioncall_output_parsers import ReActOutputJsonKeyStrParser


def get_OpenAI_chain(provider: Literal['ChatOpenAI', 'AzureChatOpenAI'], model_or_deployment_name: str):
    if provider == 'ChatOpenAI':
        chat_model = ChatOpenAI(model_name=model_or_deployment_name, temperature=0).bind(functions=[trajectory_getting_func], function_call={'name':'Trajectory'})
    else:
        chat_model = AzureChatOpenAI(deployment_name=model_or_deployment_name, temperature=0).bind(functions=[trajectory_getting_func], function_call={'name':'Trajectory'})
    return chat_model

prompt = hub.pull("jet-taekyo-lee/tagging-extracting-agent-scratchpad")
# from langchain.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages([
#     ('system', 'Think carefully, and then tag the text as instructed.\n\nTask:\n{task}'), 
#     ('human', "Agent's scratchpad:\n{scratchpad}")
# ])

provider = 'AzureChatOpenAI'
model_or_deployment_name = os.getenv('model_name') if provider == 'ChatOpenAI' else os.getenv('deployment_name')
chat_model = get_OpenAI_chain(provider, model_or_deployment_name)



chain = prompt | chat_model
chain_with_parser = prompt | chat_model | ReActOutputJsonKeyStrParser(key='observations')


### 4. Get a response

In [25]:
response = chain.invoke({'task':task, 'scratchpad':scratchpad})
print(response.additional_kwargs['function_call']['arguments'])

{
  "task": "Which is bigger? BTS's oldest member's birth month vs. incumbent Korean president's birth month.",
  "thoughts": [
    {
      "thought": "I need to find out the birth month of the oldest member of BTS and the birth month of the current Korean president."
    },
    {
      "thought": "The oldest member of BTS is Jin, and his birth month is December."
    },
    {
      "thought": "The current President of South Korea, Yoon Suk Yeol, was born in December."
    },
    {
      "thought": "The birth month of the oldest member of BTS, Jin, and the birth month of the current Korean president, Yoon Suk Yeol, are both December. Therefore, neither is bigger as they are the same."
    }
  ],
  "actions": [
    {
      "action": "GetFromYDC",
      "action_input": {
        "Query": "Oldest member of BTS birth month"
      }
    },
    {
      "action": "GetFromYDC",
      "action_input": {
        "Query": "Current Korean president birth month"
      }
    },
    {
      "action": 

In [28]:
response_with_parser = chain_with_parser.invoke({'task':task, 'scratchpad':scratchpad})
print(response_with_parser)

1.  The oldest member of BTS is Jin, and his birth month is December.
2.  The current President of South Korea, Yoon Suk Yeol, was born in December.
